# JODA 2021 Harjoitustyö - Simo Uusijoki

## Alkuvalmistelut (Kehitysympäristön valinta)

Kehtiysympäristöksi valikoitui Anaconda Navigator ja työkirjaksi tämän kautta luotu Jupyter Notebook - tiedosto. Tuodaan vaadittavat python -kirjastot osaksi työkirjaa.  

In [4]:
import pandas as pd

## Datan kerääminen

Dataksi valikoitui Inside Airbnbn data Milanosta. Alla data haettuna. 

In [27]:
import numpy as np
import matplotlib as plt
from sklearn import ensemble
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from collections import Counter

# Reading our dataset into a dataframe

URL2021= 'http://data.insideairbnb.com/italy/lombardy/milan/2021-02-17/data/listings.csv.gz'
df2021= pd.read_csv(URL2021, compression='gzip')

URL2019= 'http://data.insideairbnb.com/italy/lombardy/milan/2019-02-07/data/listings.csv.gz'
df2019= pd.read_csv(URL2019, compression='gzip')

# Check data, print columns
       
print(df2021.head())
print(df2021.columns.tolist())

print(df2019.head())
print(df2019.columns.tolist())



      id                         listing_url       scrape_id last_scraped  \
0   6400   https://www.airbnb.com/rooms/6400  20210217171707   2021-02-19   
1  17232  https://www.airbnb.com/rooms/17232  20210217171707   2021-02-19   
2  18429  https://www.airbnb.com/rooms/18429  20210217171707   2021-02-19   
3  23986  https://www.airbnb.com/rooms/23986  20210217171707   2021-02-19   
4  32119  https://www.airbnb.com/rooms/32119  20210217171707   2021-02-20   

                                 name  \
0                    The Studio Milan   
1         MPX-Center-EXPO Value/Money   
2    Best Opportunity MPX-EXPO-CENTER   
3     " Characteristic Milanese flat"   
4  Nico & Cynthia's Easy Yellow Suite   

                                         description  \
0  Enjoy your stay at The Studio, a light-filled ...   
1  convenient , easy, cozy<br />best english and ...   
2  whole apartment<br />cosy<br />convenient<br /...   
3  I look forward to welcoming you in my flat; it...   
4  <b>The 

C:\Users\simo.uusijoki\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
# Check data quantity 2021
len (df2021.index)


18367

In [32]:
# Check data quantity 2019
len (df2019.index)

17823

In [57]:
# Select columns want to be kept

cols = ['id',
    'host_response_rate',
'host_acceptance_rate',
'host_is_superhost',
'neighbourhood_cleansed',
'room_type',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'price',
'has_availability',
'availability_30',
'availability_60',
'number_of_reviews',
'review_scores_rating',
]


# read the file and make data frames to have only columns selected above

df2021 = pd.read_csv(URL2021, usecols=cols, compression='gzip')
df2019 = pd.read_csv(URL2019, usecols=cols, compression='gzip')

# Let's check if we have only wanted columns included

print(df2021.columns.tolist())
print(df2019.columns.tolist())


['id', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'has_availability', 'availability_30', 'availability_60', 'number_of_reviews', 'review_scores_rating']
['id', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'has_availability', 'availability_30', 'availability_60', 'number_of_reviews', 'review_scores_rating']


## Datan jalostaminen
Tässä osiossa jalostetaan yllä hankittua Milanon Airbnb-dataa vuosilta 2019 (ennen koronaa) ja 2021 (koronan aikana). Tässä osiossa yhdistän nuo kaksi datasettiä (molemmissa jo aiemmin samat sarakkeet) ja tutkin mitä arvoja valitut sarakkeet saavat. Sarakkeiden tietojen perusteella teen siivousta ja teen piirteiden tunnistamista.

In [54]:
# Adding column "Year" to both datasets with corresponding year values

df2021 ['Year'] = 2021
df2019 ['Year'] = 2019

In [58]:
# Let's check our columns now to see if we have new column there

print(df2021.columns.tolist())
print(df2019.columns.tolist())

['id', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'has_availability', 'availability_30', 'availability_60', 'number_of_reviews', 'review_scores_rating']
['id', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'has_availability', 'availability_30', 'availability_60', 'number_of_reviews', 'review_scores_rating']


In [59]:
# Let's check what kind of values we have within our just created columns

print(df2021.head())
print(df2019.head())

      id host_response_rate host_acceptance_rate host_is_superhost  \
0   6400                NaN                   0%                 f   
1  17232                NaN                   0%                 f   
2  18429                NaN                   0%                 f   
3  23986                NaN                 100%                 f   
4  32119               100%                  75%                 f   

  neighbourhood_cleansed        room_type  accommodates  bathrooms  bedrooms  \
0                TIBALDI     Private room             1        NaN       3.0   
1                 BOVISA      Shared room             2        NaN       1.0   
2                 BOVISA  Entire home/apt             4        NaN       2.0   
3                NAVIGLI  Entire home/apt             4        NaN       1.0   
4            VIALE MONZA  Entire home/apt             4        NaN       1.0   

   beds    price has_availability  availability_30  availability_60  \
0   1.0  $100.00           

In [ ]:
concat()